In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Flatten, LSTM
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Conv2D, Conv3D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import GlobalAveragePooling2D
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import regularizers
from tensorflow.keras.losses import categorical_crossentropy
from tensorflow.keras.losses import binary_crossentropy
from sklearn.metrics import f1_score
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras import backend as K
from keras.callbacks import EarlyStopping
from keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from PIL import Image 
import seaborn as sns
import os
import re
import glob
import cv2
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.utils.multiclass import unique_labels
from sklearn.model_selection import StratifiedKFold
from sklearn import preprocessing
import tqdm
from numpy import loadtxt
from os import *
from sklearn.utils import class_weight
import shutil
import random

Using TensorFlow backend.


In [2]:
def atoi(text):
    return int(text) if text.isdigit() else text
def natural_keys(text):
    return [atoi(c) for c in re.split('(\d+)', text)]

In [3]:
def Images_path(path_data):
    
    p = '/scratch-shared/victor/IMG_A549/'
    
    
    
    pa_adr = p + 'ADR_tile/'
    pa_hrh = p + 'HRH_tile/'
    pa_dmso = p + 'DMSO_tile/'
    
    image_list = []
    
    
       


    for filename in path_data: 
        
        if 'adr' in filename:
            

            image_list.append(pa_adr + filename)
        
            
        if 'hrh' in filename:

            image_list.append(pa_hrh + filename)
            
        if 'dmso' in filename:
            

            image_list.append(pa_dmso + filename)

    return image_list 

In [4]:
def label(y):
    lab = []
    for i in y:
        if 'adr' in i:
            lab.append(0)
        if 'hrh' in i:
            lab.append(1)
        if 'dmso' in i:
            lab.append(2)
    return lab

In [5]:
met = ['F10']
mid = ['D5']
oxy = ['F6']




cycl = ['C4']
dime =  ['F7']
cypr  = ['G9']


In [6]:
tot_well_adr = [met,mid,oxy]

tot_well_hrh = [cycl, dime, cypr]

string_well_adr = ['met', 'mid', 'oxy']

string_well_hrh = ['cycl', 'dime', 'cypr']


In [7]:
tot_well = []
string_well = [] 

In [8]:
a = 'ADR' # FOR TEST SET
b = 'HRH' # FOR REST

if a == 'HRH':
    tot_well = tot_well_hrh
    string_well = string_well_hrh
    
    
if a == 'ADR':
    tot_well = tot_well_adr
    string_well = string_well_adr

In [9]:
tot_results_accuracy = []

results_lstm = []

In [11]:
time_points = list(map(str, range(1,34)))

new_time = []
for i in time_points:
    r = '_' + i + '.'
    new_time.append(r)



path_test = '/scratch-shared/victor/IMG_A549/{}_tile/'.format(a)

# NAME OF THE WELLS CORRESPONDING TO THE DRUG THAT YOU WANT IN THE TEST SET 

wells_drug = [tot_well[0][0]] 

test = []

for _,_, filenames in os.walk(path_test):

    for filename in sorted(filenames, key = natural_keys):

        for w in wells_drug:
            for t in new_time:
                if '{}'.format(w) in filename and '{}tiff'.format(t) in filename:
                    test.append(filename)

groups_list = ['{}'.format(a), '{}'.format(b)]

fileds_of_view = ['1','2','3','4','5']

field_train, field_val = train_test_split(fileds_of_view, test_size=0.4, random_state=5)

train = []

validation = []

group_compounds = []

all_wells = tot_well_adr + tot_well_hrh
all_wells.remove(wells_drug)
allw = [j for i in all_wells for j in i]


for group in tqdm.tqdm(groups_list):

    pa = '/scratch-shared/victor/IMG_A549/{}_tile/'.format(group)

    for _,_, filenames in os.walk(pa):

        for filename in sorted(filenames, key = natural_keys):

            for t in new_time:
                
                for al in allw:

                    if '_{}-'.format(al) in filename  and '{}tiff'.format(t) in filename:

                        group_compounds.append(filename)

                    
pp = '/scratch-shared/victor/IMG_A549/DMSO_tile/'

dm = []
for _,_, filenames in os.walk(pp): 

    for f in filenames:
        dm.append(f)


group_compounds = group_compounds + dm

for i in group_compounds:

    for f in field_train:
        if '-{}_'.format(f) in i:
            train.append(i)


    for v in field_val:
        if '-{}_'.format(v) in i:
            validation.append(i)


100%|██████████| 2/2 [00:07<00:00,  3.67s/it]


In [12]:
len(train), len(validation), len(test)

(22608, 15072, 2640)

In [13]:
train_leb = label(train)
val_leb = label(validation)
test_leb = label(test)

In [14]:
train = Images_path(train)
validation = Images_path(validation)
test = Images_path(test)

In [15]:
data_train = {'id': train, 'labels':train_leb}
data_val = {'id': validation, 'labels':val_leb}

In [16]:
df_train = pd.DataFrame(data_train, columns = ['id', 'labels'])
df_val = pd.DataFrame(data_val, columns = ['id', 'labels'])

df_train['labels'] = df_train['labels'].astype(str)
df_val['labels'] = df_val['labels'].astype(str)

In [18]:
datagen=ImageDataGenerator(preprocessing_function = preprocess_input)

train_generator=datagen.flow_from_dataframe(dataframe=df_train, directory = None , x_col="id", y_col="labels",color_mode="rgb", 
                                            class_mode="categorical", target_size=(256, 256), batch_size=256)

val_generator=datagen.flow_from_dataframe(dataframe=df_val, directory = None , x_col="id", y_col="labels",color_mode="rgb", 
                                            class_mode="categorical", target_size=(256, 256), batch_size=256)

Found 22608 validated image filenames belonging to 3 classes.
Found 15072 validated image filenames belonging to 3 classes.


In [19]:
es = EarlyStopping(monitor='val_loss', mode='min', verbose=2, patience=3)

pretrained_model = VGG16(weights='imagenet',include_top=False, input_shape=(256, 256, 3))

base_model = Model(inputs=pretrained_model.input, outputs=pretrained_model.get_layer('block3_pool').output)



In [20]:
m4 = Sequential()
m4.add(base_model)


m4.add(BatchNormalization())
m4.add(GlobalAveragePooling2D())
m4.add(Dense(128, activation='relu'))
m4.add(BatchNormalization())
m4.add(Dense(64, activation='relu'))
m4.add(BatchNormalization())
m4.add(Activation('relu'))
m4.add(Dense(3,activation='softmax'))


base_model.trainable = False

opt = keras.optimizers.Adam(lr=1e-3)

m4.compile(loss= keras.losses.categorical_crossentropy, optimizer=opt, metrics = ['accuracy'])


In [21]:
epochs = 10
STEP_SIZE_TRAIN=train_generator.n//train_generator.batch_size
STEP_SIZE_VALID=val_generator.n//val_generator.batch_size

m4_h = m4.fit(train_generator, steps_per_epoch = STEP_SIZE_TRAIN, validation_data = val_generator, 
              validation_steps = STEP_SIZE_VALID , callbacks = [es], epochs=epochs, verbose = 1)

  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 88 steps, validate for 58 steps
Epoch 1/10
88/88 [==============================] - 98s 1s/step - loss: 0.7125 - accuracy: 0.6898


/home/jovyan/.local/lib/python3.6/site-packages/keras/callbacks/callbacks.py:846: RuntimeWarning: Early stopping conditioned on metric `val_loss` which is not available. Available metrics are: loss,accuracy
  (self.monitor, ','.join(list(logs.keys()))), RuntimeWarning


KeyboardInterrupt: 